<a href="https://colab.research.google.com/github/seanvw/TR_Trade_1/blob/main/Share_dealing_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To deepseek-r1-distill-llama-70b with Groq**

Write a python application to make calculations of profit/loss and APR for each Share held in a Portfolio.

Include a demonstation of it's usage.

You can use the current share price for a given ticker symbol using yfinance.

A share can be bought and sold within a day or over a longer period, months or years. It is very important that a share can be bought and sold within the same day, therefore the calaculation of days should not be an integer, but rather a floating point number.

When a share is bought, the date and time is captured to the minute.

When a share is sold, the date and time is captured to the minute.

Foreach share held within the portfolio where the sale was realized the application should calculate the realised profit/loss, the APR, together with its bought and sold datetimes and the days elapsed between purchase and sale.  

Foreach share held within the portfolio where the sale was not realized (i.e. unrealized) the application should calculate the profit/loss and APR based upon the current share price from yfinance as if the share was sold at that point in time (this can be thought of as the effective APR)

A final report should bring together realized and unrealized share dealings in a summary that calculates the grand total APR and profit/loss.

The demonstation of usage should give examples of realised share dealings held for 0.5 day, 1 day, 5 days and 6 months respectively

The demonstation of usage should give examples of unrealised share purchases

Respond with the code only! Nothing else!

The generated code was adjusted in the following ways:

*   The AMZN bought share price was changed.
*   Additional information added to reports
*   Examples changed and checked for validity of APR rates
*   More






In [ ]:

import yfinance as yf
from datetime import datetime

class Share:
    def __init__(self, ticker, quantity, buy_price, buy_datetime, sell_price=None, sell_datetime=None):
        self.ticker = ticker
        self.quantity = quantity
        self.buy_price = buy_price
        self.buy_datetime = buy_datetime
        self.sell_price = sell_price
        self.sell_datetime = sell_datetime

    def calculate_time_elapsed(self):
        if self.sell_datetime:
            time_diff = self.sell_datetime - self.buy_datetime
            days = time_diff.days + time_diff.seconds / (24 * 3600)
            return days
        else:
            current_time = datetime.now()
            time_diff = current_time - self.buy_datetime
            days = time_diff.days + time_diff.seconds / (24 * 3600)
            return days

    def calculate_apr(self):
        if self.sell_datetime:
            days = self.calculate_time_elapsed()
            profit = (self.sell_price - self.buy_price) * self.quantity
            apr = (profit / (self.buy_price * self.quantity)) / (days / 365) * 100
            return apr
        else:
            current_price = yf.Ticker(self.ticker).info['currentPrice']
            days = self.calculate_time_elapsed()
            unrealized_profit = (current_price - self.buy_price) * self.quantity
            apr = (unrealized_profit / (self.buy_price * self.quantity)) / (days / 365) * 100
            return apr

class Portfolio:
    def __init__(self):
        self.shares = []

    def add_share(self, share):
        self.shares.append(share)

    def generate_report(self):
        realized_report = []
        unrealized_report = []
        total_profit = 0
        total_apr = 0

        for share in self.shares:
            if share.sell_datetime:
                days = share.calculate_time_elapsed()
                profit = (share.sell_price - share.buy_price) * share.quantity
                apr = share.calculate_apr()
                total_profit += profit
                total_apr += apr
                realized_report.append({
                    'ticker': share.ticker,
                    'buy_price': share.buy_price,
                    'quantity': share.quantity,
                    'sell_price': share.sell_price,
                    'buy_datetime': share.buy_datetime.strftime('%Y-%m-%d %H:%M'),
                    'sell_datetime': share.sell_datetime.strftime('%Y-%m-%d %H:%M'),
                    'days': round(days, 2),
                    'profit': round(profit, 2),
                    'apr': round(apr, 2)
                })
            else:
                current_price = yf.Ticker(share.ticker).info['currentPrice']
                days = share.calculate_time_elapsed()
                unrealized_profit = (current_price - share.buy_price) * share.quantity
                apr = share.calculate_apr()
                total_profit += unrealized_profit
                total_apr += apr
                unrealized_report.append({
                    'ticker': share.ticker,
                    'buy_price': share.buy_price,
                    'quantity': share.quantity,
                    'buy_datetime': share.buy_datetime.strftime('%Y-%m-%d %H:%M'),
                    'current_price': round(current_price, 2),
                    'days': round(days, 2),
                    'unrealized_profit': round(unrealized_profit, 2),
                    'apr': round(apr, 2)
                })

        summary = {
            'total_profit': round(total_profit, 2),
            'average_apr': round(total_apr / len(self.shares) if self.shares else 0, 2)
        }

        return realized_report, unrealized_report, summary

# Demonstration of usage
portfolio = Portfolio()

# Realized shares
portfolio.add_share(Share(
    'AAPL',
    10,
    150.0,
    datetime(2023, 10, 2, 9, 30),
    300.0,
    datetime(2024, 10, 1, 9, 30)
))

portfolio.add_share(Share(
    'MSFT',
    5,
    300.0,
    datetime(2023, 10, 2, 10, 0),
    150.0,
    datetime(2023, 11, 2, 10, 0)
))

portfolio.add_share(Share(
    'GOOG',
    2,
    2500.0,
    datetime(2023, 4, 1, 10, 0),
    2501.0,
    datetime(2023, 4, 1, 22, 0)
))

# Unrealized shares
portfolio.add_share(Share(
    'AMZN',
    3,
    18.0,
    datetime(2023, 10, 1, 10, 0)
))

# Generate reports
realized, unrealized, summary = portfolio.generate_report()

# Print reports
print("Realized Transactions:")
print("=" * 30)

for transaction in realized:
    print(f"""
    Ticker: {transaction['ticker']}
    Buy
    Time: {transaction['buy_datetime']}
    Price: {transaction['buy_price']}
    Quantity: {transaction['quantity']}

    Sell
    Time: {transaction['sell_datetime']}
    Days Held: {transaction['days']}
    Price: {transaction['sell_price']}

    Realized Profit: {transaction['profit']}
    APR: {transaction['apr']}%
    """)
    print("-" * 30)

print("\nUnrealized Holdings:")
time_now = datetime.now().strftime('%Y-%m-%d %H:%M')
print(f"Current Time: {time_now}")
print("=" * 30)


for holding in unrealized:
    print(f"""
    Ticker: {holding['ticker']}

    Buy
    Time: {holding['buy_datetime']}
    Price: {holding['buy_price']}
    Quantity: {holding['quantity']}

    Not Sold
    Current Price: {holding['current_price']}
    Days Held: {holding['days']}

    Unrealized Profit: {holding['unrealized_profit']}
    APR: {holding['apr']}%
    """)

print("\nPortfolio Summary:")
print(f"Total Profit: {summary['total_profit']}")
print(f"Average APR: {summary['average_apr']}%")

Realized Transactions:

    Ticker: AAPL
    Buy      
    Time: 2023-10-02 09:30
    Price: 150.0
    Quantity: 10

    Sell   
    Time: 2024-10-01 09:30
    Days Held: 365.0
    Price: 300.0
    
    Realized Profit: 1500.0
    APR: 100.0%
    
------------------------------

    Ticker: MSFT
    Buy      
    Time: 2023-10-02 10:00
    Price: 300.0
    Quantity: 5

    Sell   
    Time: 2023-11-02 10:00
    Days Held: 31.0
    Price: 150.0
    
    Realized Profit: -750.0
    APR: -588.71%
    
------------------------------

    Ticker: GOOG
    Buy      
    Time: 2023-04-01 10:00
    Price: 2500.0
    Quantity: 2

    Sell   
    Time: 2023-04-01 22:00
    Days Held: 0.5
    Price: 2501.0
    
    Realized Profit: 2.0
    APR: 29.2%
    
------------------------------

Unrealized Holdings:
Current Time: 2025-03-07 17:50

    Ticker: AMZN
          
    Buy
    Time: 2023-10-01 10:00
    Price: 18.0
    Quantity: 3    
    
    Not Sold 
    Current Price: 195.82
    Days Held: 5